In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

In [16]:
far.utils._check()

4


In [17]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])

In [18]:
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

_et = far.get_hyperparameter('eta', 0.1)
io_optim = far.MomentumOptimizer(_et, 
                                   far.get_hyperparameter('mu', .9))

io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)

optim_oo = tf.train.AdamOptimizer(.01)

In [19]:
far.utils.hyperparameters()

[<tf.Variable 'lambda_components/0_0:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_components/0_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_components/1_0:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_components/1_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

In [20]:
io_optim_dict.state

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable/Momentum:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Momentum:0' shape=(2, 2) dtype=float32_ref>]

In [21]:
farho = far.HyperOptimizer(hypergradient=far.ForwardHG())
run = farho.minimize(oo, optim_oo, cost, io_optim, hyper_list=[_et])

[<tf.Tensor 'Mean_1_1/gradients/mul_5_grad/Reshape:0' shape=(2,) dtype=float32>, None, <tf.Tensor 'Mean_1_1/gradients/mul_5_grad/Reshape_1:0' shape=(2, 2) dtype=float32>, None]
[<tf.Variable 'Mean_1/Z/Variable/eta:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'Mean_1/Z/Variable/Momentum/eta:0' shape=(2,) dtype=float32_ref>, <tf.Variable 'Mean_1/Z/Variable_1/eta:0' shape=(2, 2) dtype=float32_ref>, <tf.Variable 'Mean_1/Z/Variable_1/Momentum/eta:0' shape=(2, 2) dtype=float32_ref>]
[<tf.Tensor 'Mean_1_1/mul:0' shape=(2,) dtype=float32>, <tf.Tensor 'Mean_1_1/mul_1:0' shape=(2, 2) dtype=float32>]


ValueError: Shapes must be equal rank, but are 1 and 2
	From merging shape 0 with other shapes. for 'Mean_1_1/Rank/packed' (op: 'Pack') with input shapes: [2], [2,2].

In [ ]:
farho.

In [147]:
only_dy = [v[1] for v in io_optim_dict.dynamics]

In [148]:
aux_v = [tf.zeros_like(v) for v in only_dy]

In [149]:
eta = far.utils.hyperparameters()[1]

In [150]:
## try to compute forward automatically.... far all scalar hyperparameters
g_all_hypers = tf.gradients(far.utils.dot(far.utils.vectorize_all(only_dy), far.utils.vectorize_all(aux_v)),
                           far.utils.hyperparameters())
print(g_all_hypers)
B_ts = [tf.gradients(g_eta, aux_v) for g_eta in g_all_hypers]

[<tf.Tensor 'gradients_1/gradients/mul_grad/mul_1_grad/Reshape:0' shape=(2, 2) dtype=float32>, <tf.Tensor 'gradients_1/AddN_1:0' shape=() dtype=float32>, <tf.Tensor 'gradients_1/AddN_3:0' shape=() dtype=float32>]


In [151]:
g_all_hypers

[<tf.Tensor 'gradients_1/gradients/mul_grad/mul_1_grad/Reshape:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'gradients_1/AddN_1:0' shape=() dtype=float32>,
 <tf.Tensor 'gradients_1/AddN_3:0' shape=() dtype=float32>]

In [152]:
B_ts

[[<tf.Tensor 'gradients_2/Vectorization_1/Reshape_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_2/Vectorization_1/Reshape_1_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_2/Vectorization_1/Reshape_2_grad/Reshape:0' shape=(2, 2) dtype=float32>,
  <tf.Tensor 'gradients_2/Vectorization_1/Reshape_3_grad/Reshape:0' shape=(2, 2) dtype=float32>],
 [<tf.Tensor 'gradients_3/Vectorization_1/Reshape_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_3/Vectorization_1/Reshape_1_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_3/Vectorization_1/Reshape_2_grad/Reshape:0' shape=(2, 2) dtype=float32>,
  <tf.Tensor 'gradients_3/Vectorization_1/Reshape_3_grad/Reshape:0' shape=(2, 2) dtype=float32>],
 [<tf.Tensor 'gradients_4/Vectorization_1/Reshape_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_4/Vectorization_1/Reshape_1_grad/Reshape:0' shape=(2,) dtype=float32>,
  <tf.Tensor 'gradients_4/Vectorization_1/Resh

In [153]:
tf.global_variables_initializer().run()

In [154]:
ss.run(B_ts)

[[array([ 0.,  0.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 0.2       ,  0.40000001],
         [-0.2       ,  0.        ]], dtype=float32),
  array([[-2., -4.],
         [ 2.,  0.]], dtype=float32)],
 [array([-1., -3.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 2.,  4.],
         [-2.,  0.]], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32)],
 [array([ 0.,  0.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32)]]

In [143]:
ss.run(B_ts)

[[array([ 0.,  0.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 0.2       ,  0.40000001],
         [-0.2       ,  0.        ]], dtype=float32),
  array([[-2., -4.],
         [ 2.,  0.]], dtype=float32)],
 [array([-1., -3.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 2.,  4.],
         [-2.,  0.]], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32)],
 [array([ 0.,  0.], dtype=float32),
  array([ 0.,  0.], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32),
  array([[ 0.,  0.],
         [ 0.,  0.]], dtype=float32)]]

In [ ]:
## try to compute forward automatically.... far all scalar hyperparameters
dots_with_dummy_v = [far.utils.dot(dy, v) for dy, v in zip(only_dy, aux_v)]

g_all_hypers = tf.gradients(far.utils.dot(far.utils.vectorize_all(only_dy), far.utils.vectorize_all(aux_v)),
                           far.utils.hyperparameters())
print(g_all_hypers)
B_ts = [tf.gradients(g_eta, aux_v) for g_eta in g_all_hypers]

In [64]:
shape = (2, 3, 4, 1, 5)
# for sh in np.shape:
#     print(i)

In [65]:
np.multiply.reduce(shape)

120

In [66]:
a = np.zeros(shape)
for k in range(np.multiply.reduce(shape)):
    indices = np.unravel_index(k, shape) 
    print(indices)
    print('_'.join([str(ind) for ind in indices]))
    a[indices] = k

(0, 0, 0, 0, 0)
0_0_0_0_0
(0, 0, 0, 0, 1)
0_0_0_0_1
(0, 0, 0, 0, 2)
0_0_0_0_2
(0, 0, 0, 0, 3)
0_0_0_0_3
(0, 0, 0, 0, 4)
0_0_0_0_4
(0, 0, 1, 0, 0)
0_0_1_0_0
(0, 0, 1, 0, 1)
0_0_1_0_1
(0, 0, 1, 0, 2)
0_0_1_0_2
(0, 0, 1, 0, 3)
0_0_1_0_3
(0, 0, 1, 0, 4)
0_0_1_0_4
(0, 0, 2, 0, 0)
0_0_2_0_0
(0, 0, 2, 0, 1)
0_0_2_0_1
(0, 0, 2, 0, 2)
0_0_2_0_2
(0, 0, 2, 0, 3)
0_0_2_0_3
(0, 0, 2, 0, 4)
0_0_2_0_4
(0, 0, 3, 0, 0)
0_0_3_0_0
(0, 0, 3, 0, 1)
0_0_3_0_1
(0, 0, 3, 0, 2)
0_0_3_0_2
(0, 0, 3, 0, 3)
0_0_3_0_3
(0, 0, 3, 0, 4)
0_0_3_0_4
(0, 1, 0, 0, 0)
0_1_0_0_0
(0, 1, 0, 0, 1)
0_1_0_0_1
(0, 1, 0, 0, 2)
0_1_0_0_2
(0, 1, 0, 0, 3)
0_1_0_0_3
(0, 1, 0, 0, 4)
0_1_0_0_4
(0, 1, 1, 0, 0)
0_1_1_0_0
(0, 1, 1, 0, 1)
0_1_1_0_1
(0, 1, 1, 0, 2)
0_1_1_0_2
(0, 1, 1, 0, 3)
0_1_1_0_3
(0, 1, 1, 0, 4)
0_1_1_0_4
(0, 1, 2, 0, 0)
0_1_2_0_0
(0, 1, 2, 0, 1)
0_1_2_0_1
(0, 1, 2, 0, 2)
0_1_2_0_2
(0, 1, 2, 0, 3)
0_1_2_0_3
(0, 1, 2, 0, 4)
0_1_2_0_4
(0, 1, 3, 0, 0)
0_1_3_0_0
(0, 1, 3, 0, 1)
0_1_3_0_1
(0, 1, 3, 0, 2)
0_1_3_0_2
(0, 1, 3, 0,

In [67]:
isinstance(lmbd.shape, tf.TensorShape)

True

In [68]:
a

array([[[[[   0.,    1.,    2.,    3.,    4.]],

         [[   5.,    6.,    7.,    8.,    9.]],

         [[  10.,   11.,   12.,   13.,   14.]],

         [[  15.,   16.,   17.,   18.,   19.]]],


        [[[  20.,   21.,   22.,   23.,   24.]],

         [[  25.,   26.,   27.,   28.,   29.]],

         [[  30.,   31.,   32.,   33.,   34.]],

         [[  35.,   36.,   37.,   38.,   39.]]],


        [[[  40.,   41.,   42.,   43.,   44.]],

         [[  45.,   46.,   47.,   48.,   49.]],

         [[  50.,   51.,   52.,   53.,   54.]],

         [[  55.,   56.,   57.,   58.,   59.]]]],



       [[[[  60.,   61.,   62.,   63.,   64.]],

         [[  65.,   66.,   67.,   68.,   69.]],

         [[  70.,   71.,   72.,   73.,   74.]],

         [[  75.,   76.,   77.,   78.,   79.]]],


        [[[  80.,   81.,   82.,   83.,   84.]],

         [[  85.,   86.,   87.,   88.,   89.]],

         [[  90.,   91.,   92.,   93.,   94.]],

         [[  95.,   96.,   97.,   98.,   99.]]],


        

In [161]:
np.any([None, [None, None]])

[None, None]

In [157]:
if lmbd is None:
    print(1)

In [69]:
a.shape

(2, 3, 4, 1, 5)

In [70]:
from functools import reduce

In [75]:
res = reduce(lambda a, v: [a]*v, shape[::-1], None)

In [78]:
indices = (1,1, 1, 0, 1)

In [77]:
res[(1,1, 1, 0, 1)]

SyntaxError: invalid syntax (<ipython-input-77-283a6d95f30d>, line 1)

In [84]:
a = [
    [tf.get_variable('11', initializer=1.), tf.get_variable('21', initializer=0.)]
]

In [85]:
np.array(a)

array([[<tf.Variable '11:0' shape=() dtype=float32_ref>,
        <tf.Variable '21:0' shape=() dtype=float32_ref>]], dtype=object)

In [114]:
c = tf.convert_to_tensor(np.array(a, dtype=tf.Variable))

TypeError: Expected binary or unicode string, got None

In [92]:
a = np.empty(shape, tf.Variable)

In [95]:
a[indices] = tf.get_variable('dckan', initializer=1.)

In [96]:
a

array([[[[[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]]],


        [[[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]]],


        [[[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]]]],



       [[[[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]]],


        [[[None, None, None, None, None]],

         [[None, <tf.Variable 'dckan:0' shape=() dtype=float32_ref>, None,
           None, None]],

         [[None, None, None, None, None]],

         [[None, None, None, None, None]]],


        [[[None, None, None, None, None]],

       

In [115]:
def get_hyperparameter(name, initializer=None, shape=None, dtype=None, scalar=False):
    """
    Creates an hyperparameter variable, which is a GLOBAL_VARIABLE
    and HYPERPARAMETER. Mirrors the behavior of `tf.get_variable`.

    :param name: name of this hyperparameter
    :param initializer: initializer or initial value (can be np.array or float)
    :param shape: optional shape, may be not needed depending on initializer
    :param dtype: optional type,  may be not needed depending on initializer
    :param scalar: if True divides the hyperparameter in its scalar components (use with `ForwardHG`)

    :return: the newly created variable.
    """
    if not scalar:
        return tf.get_variable(name, shape, dtype, initializer, trainable=False,
                               collections=HYPERPARAMETERS_COLLECTIONS)
    else:
        with tf.variable_scope(name + '_components'):
            _shape = shape or initializer.shape
            if isinstance(_shape, tf.TensorShape):
                _shape = _shape.as_list()
            # _tmp_lst = reduce(lambda a, v: [a]*v, shape[::-1], None)
            _tmp_lst = np.empty(_shape, object)
            for k in range(np.multiply.reduce(_shape)):
                indices = np.unravel_index(k, _shape)
                # print(indices)
                _ind_name = '_'.join([str(ind) for ind in indices])
                _tmp_lst[indices] = tf.get_variable(_ind_name, (), dtype,
                                                    initializer if callable(initializer) else initializer[indices],
                                                    trainable=False, collections=HYPERPARAMETERS_COLLECTIONS)
        return tf.convert_to_tensor(_tmp_lst.tolist(), name=name)


In [116]:
HYPERPARAMETERS_COLLECTIONS = (far.GraphKeys.HYPERPARAMETERS, far.GraphKeys.GLOBAL_VARIABLES)

In [118]:
lmbd = get_hyperparameter('lambda_11121', initializer=tf.random_normal_initializer(stddev=10.), shape=shape, scalar=True)

In [120]:
lmbd

<tf.Tensor 'lambda_11121:0' shape=(2, 3, 4, 1, 5) dtype=float32>

In [123]:
far.utils.hyperparameters()

[<tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_0_0_0:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_0_0_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_0_0_2:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_0_0_3:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_0_0_4:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_1_0_0:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_1_0_1:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_1_0_2:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_1_0_3:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_1_0_4:0' shape=() dtype=float32_ref>,
 <tf.Variable 'lambda_1_components/0_0_2_0_0:0' shape=() dtype=float32_

In [125]:
tf.global_variables_initializer().run()

In [126]:
lmbd.eval()

array([[[[[ 13.63513184,  14.00071239,   7.39320803,  -0.74280453,
             9.5134182 ]],

         [[ -8.83658886, -13.9394207 , -12.10483456,  -5.75274944,
            10.6792841 ]],

         [[  9.53555298,   6.12478638,   7.11979389,  -3.60314012,
            -8.63583946]],

         [[ -2.39586926,  12.63607216,   3.63382721,  18.8909111 ,
             0.49042448]]],


        [[[ -6.85972214,  11.65580845,  -2.73105001,  -7.0301199 ,
            26.33629608]],

         [[  1.86420655,  -1.69461322,   3.97628689,   0.17976785,
           -19.1612606 ]],

         [[ -2.68925381,   3.14724493,   5.15794992,  -7.49233532,
           -16.58163834]],

         [[ 10.82481956,  -5.98878765, -15.80984211,  15.24536896,
            -2.77124619]]],


        [[[ -8.06270981,   7.08530235,  -3.35549355,   1.07518125,
            -5.39646864]],

         [[-17.54975128,  13.82890701,  10.00484276,  11.93820953,
            -9.01740646]],

         [[ -1.03552425, -13.41908073,   3.101

In [119]:
c = tf.convert_to_tensor(lmbd)

In [73]:
# this look quite right...

In [74]:
tf.global_variables()

[<tf.Variable 'Variable:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable/Momentum:0' shape=(2,) dtype=float32_ref>,
 <tf.Variable 'Variable_1/Momentum:0' shape=(2, 2) dtype=float32_ref>]

In [303]:
far.utils.hyperparameters()

[<tf.Variable 'lambda:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'eta:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu:0' shape=() dtype=float32_ref>]

In [304]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [305]:
rhg.run(10)

In [306]:
ss.run(far.utils.hypergradients())

[array([[ 0.22334358,  1.34006155],
        [-0.22334358,  0.        ]], dtype=float32), 13.395197, 0.28557456]

In [307]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))

# check with rfho

In [308]:
import rfho as rf

In [309]:
w, c, co = rf.vectorize_model([v1, v2], cost, oo, augment=1)

In [310]:
dyn = rf.MomentumOptimizer.create(w, 0.1, 0.9, loss=c)

In [311]:
hyperg = rf.HyperOptimizer(dyn, {co: lmbd}, rf.ReverseHG)

In [312]:
hyperg.initialize()

True

In [313]:
hgs = hyperg.run(10, val_feed_dict_suppliers={co:lambda: None},
                 _debug_no_hyper_update=True)

In [314]:
ss.run(hyperg.hyper_gradients.hyper_gradient_vars)

[array([[ 0.2233436 ,  1.34006178],
        [-0.2233436 ,  0.        ]], dtype=float32)]

# END... RESULTS ARE (almost the) SAME (CFR cell 306)

In [209]:
rfho_hs = hyperg.hyper_gradients.w_hist

In [210]:
len(rfho_hs)

10

In [218]:
rfho_hs[9], rhg._history[10]  # in far version also last value is saved

(array([-24.56974983, -22.56974983, -21.56974983,   0.27419662,
          0.54839325,  -0.27419662,   0.        ,  44.92248917,
         44.92248917,  44.92248917,   2.48619032,   4.97238111,
         -2.48619032,   0.        ], dtype=float32),
 [array([-29.34610748, -27.34610748, -26.34610748], dtype=float32),
  array([ 47.76357651,  47.76357651,  47.76357651], dtype=float32),
  array([[-0.00439989, -0.00879972],
         [ 0.00439989,  0.        ]], dtype=float32),
  array([[ 2.78596425,  5.57192898],
         [-2.78596425,  0.        ]], dtype=float32)])

In [219]:
[1,2,4][:-1]

[1, 2]

In [222]:
ss.run(list(hyperg.hyper_gradients.p_dict.values()))

[array([  7.33333349e+00,   7.33333349e+00,   7.33333349e+00,
         -3.87176406e-05,  -7.74352811e-05,   3.87176406e-05,
          0.00000000e+00,  -4.29872227e+00,  -4.29872227e+00,
         -4.29872227e+00,  -1.10320083e-03,  -2.20641680e-03,
          1.10320083e-03,   0.00000000e+00], dtype=float32)]